In [533]:
day = 17
from IPython.display import IFrame
url = "https://adventofcode.com/2022/day/%d"%(day)
IFrame(src=url, width=1000, height=600)

In [534]:
import urllib.request
import sys
from timeit import default_timer as timer
import itertools


def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2022):
    with open('sessionID') as f:
        sessionId = f.readlines()[0]
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data

def read_file(filename):
    # read file
    with open(filename) as f:
        lines = f.readlines()
    output = [str.rstrip(line) for line in lines]    
    return output



In [535]:
import numpy as np

def init():
    wind = np.array([1 if val=='>' else -1 for val in data])
    rocks = [np.expand_dims(np.array([1,1,1,1]),axis=0),\
             np.array([[0,1,0],[1,1,1],[0,1,0]]),\
             np.array([[0,0,1],[0,0,1],[1,1,1]]),\
             np.array([[1],[1],[1],[1]]),\
             np.array([[1,1],[1,1]]) ]
    rocks = np.array(rocks, dtype='object')
    chamber = np.zeros([0,7])
    return chamber, rocks, wind


def rock_in_chamber(rock,row,col):
    r,c = rock.shape
    return chamber[row:row+r,col:col+c] 

def add_rock_in_chamber(rock, row, col):
    r,c = rock.shape
    #print(chamber[row:row+r,col:col+c].shape, rock.shape)
    chamber[row:row+r,col:col+c] += rock
    non_zero_row = np.where(chamber.sum(axis=1)==0)
    return non_zero_row[0][-1] 

In [574]:
## start of sim
data = get_data(day)[0]
data = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'
chamber, rocks, wind = init()

## start of rock
for count in range(2023):
    rock = rocks[0]
    rocks = np.roll(rocks, -1)
    col = 2
    row = -1
    chamber = np.vstack([np.zeros([rock.shape[0]+3,7]), chamber])
    
    #print(rock.shape,'\n',chamber,'\n')
    
    drop = True
    while drop:
        row += 1
        new = rock_in_chamber(rock,row,col)
        drop = (not new.any()) & (new.size!=0) & (new.shape==rock.shape)
        #print(row,col)
        if drop:
            #print(row,col)
            delta = wind[0]
            wind = np.roll(wind, -1)
            #print(delta)
            new = rock_in_chamber(rock, row, col+delta)
            #print(new.any(), delta)
            if (new.shape==rock.shape) & (not new.any()):
                col += delta
            
    max_row = add_rock_in_chamber(rock, row-1, col)
    chamber = chamber[max_row+1:,:]
    print(count+1, chamber.shape[0])
    if count == 9:
        break
    
    


1 1
2 4
3 6
4 7
5 9
6 10
7 13
8 15
9 17
10 17


In [575]:
px.imshow(chamber)


In [577]:
import sys
import math
from copy import deepcopy
from collections import defaultdict, deque
data = get_data(17)[0]
data = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

def get_piece(t, y): # set of (x,y) pairs
    A = set()
    if t==0:
        return set([(2,y), (3,y), (4,y), (5,y)])
    elif t == 1:
        return set([(3, y+2), (2, y+1), (3,y+1), (4,y+1), (3,y)])
    elif t == 2:
        return set([(2, y), (3,y), (4,y), (4,y+1), (4,y+2)])
    elif t==3:
        return set([(2,y),(2,y+1),(2,y+2),(2,y+3)])
    elif t==4:
        return set([(2,y+1),(2,y),(3,y+1),(3,y)])
    else:
        assert False

def move_left(piece):
    if any([x==0 for (x,y) in piece]):
        return piece
    return set([(x-1,y) for (x,y) in piece])

def move_right(piece):
    if any([x==6 for (x,y) in piece]):
        return piece
    return set([(x+1,y) for (x,y) in piece])
def move_down(piece):
    return set([(x,y-1) for (x,y) in piece])
def move_up(piece):
    return set([(x,y+1) for (x,y) in piece])
def show(R):
    maxY = max([y for (x,y) in R])
    for y in range(maxY,0,-1):
        row = ''
        for x in range(7):
            if (x,y) in R:
                row += '#'
            else:
                row += ' '
        print(row)

R = set([(x,0) for x in range(7)])

def signature(R):
    maxY = max([y for (x,y) in R])
    return frozenset([(x,maxY-y) for (x,y) in R if maxY-y<=30])

L = 1000000000000

SEEN = {}
top = 0
i = 0
t = 0
added = 0
while t<L:
    #print(t, len(SEEN))
    piece = get_piece(t%5, top+4)
    while True:
        # pushed -> down
        if data[i]=='<':
            piece = move_left(piece)
            if piece & R:
                piece = move_right(piece)
        else:
            piece = move_right(piece)
            if piece & R:
                piece = move_left(piece)
        i = (i+1)%len(data)
        piece = move_down(piece)
        if piece & R:
            piece = move_up(piece)
            R |= piece
            top = max([y for (x,y) in R])

            SR = (i, t%5, signature(R))
            if SR in SEEN and t>=2022:
                (oldt, oldy) = SEEN[SR]
                dy = top-oldy
                dt = t-oldt
                amt = (L-t)//dt
                added += amt*dy
                t += amt*dt
                assert t<=L
            SEEN[SR] = (t,top)
            #show(R)
            break
    t += 1
    #print(t,top)
    if t==2022:
        print(top)
print(top+added)

3068
1514285714288
